In [ ]:
import glob
import re
import os

import xml.etree.ElementTree as ET

PREFIX = "{http://www.tei-c.org/ns/1.0}"
AUTHOR_ID = f"{PREFIX}author"
TITLE_CLEAN = re.compile('[^a-zA-Z\s]')

def first_child(parent, child_part):
  return parent.findall(f'{PREFIX}{child_part}')[0]

def get_docs(filename):
  filedesc_node = first_child(first_child(ET.parse(filename).getroot(), 'teiHeader'), 'fileDesc')
  title = first_child(first_child(filedesc_node, 'titleStmt'),  'title').text
  title = TITLE_CLEAN.sub('', title)
  if re.match('[^a-z]+$', title):
      title = title.title()
  title = "_".join(title.split())
  authors = first_child(first_child(first_child(filedesc_node, 'sourceDesc'),  'biblStruct'), 'analytic').findall(AUTHOR_ID)

  assert filename.endswith('.tei.xml')
  file_desc = filename.split('/')[-1][:-8]
  print(file_desc)
  if re.match('[0-9]+\..*', file_desc):
      if re.match('^[^a-z]+$', file_desc):
          date = f'20{file_desc[:2]}'
      else:
          date = file_desc[:4]
  else: 
    date = None
    
  last_names = [first_child(first_child(author, 'persName'), 'surname').text for author in authors]
  if not(last_names):
      author_str = "no_author"
  elif len(last_names) == 1:
      author_str, = last_names
  elif len(last_names) == 2:
      author_str = "_and_".join(last_names)
  else:
      author_str = last_names[0]+"_etal"

  if date is None:
      return f'{author_str}_{title}.pdf'
  else:
      return f'{author_str}_{date}_{title}.pdf'
    


In [ ]:
for filename in glob.glob(f"xmls/*.xml"):
  new_filename = get_docs(filename)
  pdf_filename = filename.replace('xml', 'pdf').replace('.tei', '')
  head, tail = os.path.split(pdf_filename)
  print(head, tail)
  os.rename(pdf_filename, os.path.join(head, new_filename))